# Run custom functions(运行自定义函数)

您可以在管道中使用任意函数。
请注意，这些函数的所有输入都需要是一个单一参数。如果您有一个接受多个参数的函数，您应该编写一个包装器来接受单个输入并将其解压成多个参数

In [1]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

# 获取单文本文本长度
def length_function(text):
  return len(text)

# 获取多文本文本长度
def _multi_length_function(text1, text2):
  return len(text1) + len(text2)

# 获取多文本文本长度
def _multi_length_function(_dict):
  return len(_dict["text1"]) + len(_dict["text2"])

# 定义会话基础模板
promt = ChatPromptTemplate.from_template("{a}+{b} 等于多少？")

# 创建AI模型
model = ChatOpenAI(

)

base_chain = promt | model

chain = (
  {
    "a": itemgetter("foo") | RunnableLambda(length_function),
     "b": {"text1":itemgetter("foo"), "text2":itemgetter("bar")} | RunnableLambda(_multi_length_function),
  } | base_chain
)

chain.invoke({"foo":"bar", "bar":"abcd"})




AIMessage(content='3 + 7 = 10.')

### Accepting a Runnable Config(接收一个可运行的配置)
Runnable lambda 可以选择接受 RunnableConfig，它们可以使用它来将回调、标签和其他配置信息传递给嵌套运行。


In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig

import json

def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "修复以下json数据:\n\n```text\n{input}\n```\nError: {error}"
            " 不要描述, 仅回答修复后的数据就行."
        )
        | model
        | StrOutputParser()
    )
    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
  out = RunnableLambda(parse_or_fix).invoke(
    "{foo: bar}",{"tags": ["my-tag"], "callbacks": [cb]}
  )
  print(out)
  print(cb)

{'foo': 'bar'}
Tokens Used: 69
	Prompt Tokens: 63
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $0.00010650000000000001
